In [2]:
import common_23 as common #This file works in Python 3. video works in python2
import video_23 as video #This file works in Python 3. video works for python2
import os
import numpy as np
import cv2
import random
import time 
import sys
from contextlib import contextmanager
import itertools as it
from os.path import isfile, join


In [9]:
#Convert file folder to video if you have frame sequences instead of a video
def fr2vid(inputpath, outputfile):
    pathIn= inputpath
    pathOut = outputfile
    fps = 32
    frame_array = []
    files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
    #for sorting the file names properly
    files.sort(key = lambda x: x[5:-4])
    files.sort()
    frame_array = []
    files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
    #for sorting the file names properly
    files.sort(key = lambda x: x[5:-4])
    for i in range(len(files)):
        filename=pathIn + files[i]
        #reading each files
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
    
        #inserting the frames into an image array
        frame_array.append(img)
    out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
    for i in range(len(frame_array)):
        # writing to a image array
        out.write(frame_array[i])
    out.release()

#fr2vid('./house/','house.avi') #convert sequence to video


In [2]:
# Homework 3 -- Motion Segmentation helper code

def motion_seg(inputfile):
    cap = video.create_capture(inputfile)
    ret, first = cap.read()

    # For background subtraction, Save the first image as reference
    first_gray = cv2.cvtColor(first, cv2.COLOR_BGR2GRAY)
    first_gray = cv2.GaussianBlur(first_gray, (21, 21), 0)
    start = time.time()

    while True:
        ret, frame = cap.read()
        if not ret:
            break
    
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (21, 21), 0)

        # In each iteration, calculate absolute difference between current frame and reference frame
        difference = cv2.absdiff(gray, first_gray)

        # Apply thresholding to eliminate noise
        thresh = cv2.threshold(difference, 25, 255, cv2.THRESH_BINARY)[1]
        thresh = cv2.dilate(thresh, None, iterations=5)
        
        cv2.imshow("thresh", thresh)
        key = cv2.waitKey(1) & 0xFF 
        # if the `q` key is pressed, break from the lop
        if key == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()
    print(time.time()-start)

In [21]:
motion_seg('railway.avi')

0.9704020023345947


**Non Adaptive frame differencing**

In this type of frame differencing, the background model is replaced with that of the previous image at each iteration. It is very quick to adapt to changes in lighting. Objects that stop are no longer detected. Objects that start up do not leave behind the ghosts. Very difficult to detect an object moving towards or away from the camera.  

Thresholds = [0,5,10,15,20,25,30]

As the threshold value increases, the motion of the objects and the object itself gets insignificant. Even low threshold values do not show significant motion. Threshold value of 10 seems optimal. 

In [30]:
# Non - Adaptive Backgrounding
def nonAdaptiveFD(inputfile,threshold):
    cap = video.create_capture(inputfile)
    ret, first = cap.read()

    first_gray = cv2.cvtColor(first, cv2.COLOR_BGR2GRAY)
    first_gray = cv2.GaussianBlur(first_gray, (21, 21), 0)
    start = time.time()
    backgroundFrame = first_gray
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
    
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (21, 21), 0)
        currentFrame = gray
        
        foreground = cv2.absdiff(backgroundFrame, currentFrame)

        foreground = cv2.threshold(foreground, threshold, 255, cv2.THRESH_BINARY)[1]
        foreground = cv2.dilate(foreground, None, iterations=5)
        
        cv2.imshow("Non Adaptive FD threshold {0}".format(threshold), foreground)
        
        backgroundFrame = currentFrame
        key = cv2.waitKey(100) & 0xFF 
    
        if key == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()
    print('Times to complete for threshold {0} is {1}'.format(threshold, time.time()-start))

In [32]:
#Ideal case
nonAdaptiveFD('beach.avi', 10)

Times to complete for threshold 10 is 2.6158246994018555


In [31]:
thresholds = [0,5,10,15,20,25,30]
sequences = ['beach.avi', 'trees.avi', 'lights.avi', 'jug.avi', 'railway.avi', 'rock.avi']
for sequence in sequences:
    for threshold in thresholds:
        nonAdaptiveFD(sequence,threshold)    

Times to complete for threshold 0 is 2.3468198776245117
Times to complete for threshold 5 is 0.6679394245147705
Times to complete for threshold 10 is 0.2732706069946289
Times to complete for threshold 15 is 0.2902240753173828
Times to complete for threshold 20 is 0.32058119773864746
Times to complete for threshold 25 is 0.4603567123413086
Times to complete for threshold 30 is 0.4360504150390625
Times to complete for threshold 0 is 0.6962041854858398
Times to complete for threshold 5 is 0.47173523902893066
Times to complete for threshold 10 is 0.4597747325897217
Times to complete for threshold 15 is 0.40974855422973633
Times to complete for threshold 20 is 0.4253976345062256
Times to complete for threshold 25 is 0.44284582138061523
Times to complete for threshold 30 is 0.6936357021331787
Times to complete for threshold 0 is 0.4062070846557617
Times to complete for threshold 5 is 0.1874995231628418
Times to complete for threshold 10 is 0.1984710693359375
Times to complete for threshold 1

**Adaptive frame differencing**

In this type of frame differencing, the current image is blended into the background model with parameter α (blending parameter). 

If α = 1, it becomes the Non Adaptive frame differencing whereas if α = 0, it gives the simple background subtraction. 

It is more responsive to changes in illumination and camera motion. Objects that stop and ghosts left behind by the objects that start, gradually fade into the background. 

As alpha decreases, the time taken for ghosts to disappear increases.

alphas = [1, 0.75, 0.5, 0.25, 0]

thresholds = [0,5,10,15,20,25,30]

Alpha of 0.5 and Threshold of 10 usually give good results

In [34]:
# Adaptive Backgrounding
def adaptiveFD(inputfile,alpha,threshold):
    cap = video.create_capture(inputfile)
    ret, first = cap.read()
    
    first_gray = cv2.cvtColor(first, cv2.COLOR_BGR2GRAY)
    first_gray = cv2.GaussianBlur(first_gray, (21, 21), 0)
    start = time.time()
    backgroundFrame = first_gray
    i = 1
    while True:
        ret, frame = cap.read()
        if not ret:
            break
    
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (21, 21), 0)
        currentFrame = gray
        
        foreground = cv2.absdiff(backgroundFrame, currentFrame)

        foreground = cv2.threshold(foreground, threshold, 255, cv2.THRESH_BINARY)[1]
        foreground = cv2.dilate(foreground, None, iterations=5)
            
        cv2.imshow("Adaptive FD alpha {0} threshold {1}".format(alpha, threshold), foreground)
         
        backgroundFrame = cv2.addWeighted(currentFrame, alpha, backgroundFrame, 1.0-alpha,0)
        key = cv2.waitKey(100) & 0xFF 
         
        if key == ord("q"):
            break
            
    cap.release()
    cv2.destroyAllWindows()
    print('Times to complete for alpha {0} threshold {1} is {2}'.format(alpha, threshold, time.time()-start))

In [35]:
# Ideal case
adaptiveFD('beach.avi',0.5,10)

Times to complete for alpha 0.5 threshold 10 is 2.546170949935913


In [40]:
alphas = [1, 0.75, 0.5, 0.25, 0]
thresholds = [0,5,10,15,20,25,30]
sequences = ['beach.avi', 'trees.avi', 'lights.avi', 'jug.avi', 'railway.avi', 'rock.avi']
for sequence in sequences:
    for alpha in alphas:
        for threshold in thresholds:
            adaptiveFD(sequence, alpha,threshold)

Times to complete for alpha 1 threshold 0 is 1.3493905067443848
Times to complete for alpha 1 threshold 5 is 0.12166953086853027
Times to complete for alpha 1 threshold 10 is 0.12665748596191406
Times to complete for alpha 1 threshold 15 is 0.5116353034973145
Times to complete for alpha 1 threshold 20 is 0.13364434242248535
Times to complete for alpha 1 threshold 25 is 0.12505531311035156
Times to complete for alpha 1 threshold 30 is 0.10571813583374023
Times to complete for alpha 0.75 threshold 0 is 0.10870957374572754
Times to complete for alpha 0.75 threshold 5 is 0.11070680618286133
Times to complete for alpha 0.75 threshold 10 is 0.9714035987854004
Times to complete for alpha 0.75 threshold 15 is 0.5192549228668213
Times to complete for alpha 0.75 threshold 20 is 0.09973406791687012
Times to complete for alpha 0.75 threshold 25 is 0.0827796459197998
Times to complete for alpha 0.75 threshold 30 is 0.10970687866210938
Times to complete for alpha 0.5 threshold 0 is 0.111701726913452

**Persistant frame differencing**

It is responsive to changes in illumination and camera motion, the ghosts fade away. Objects leave behind the fading trails of pixels. The gradient of this trail indicated the apparent direction of object motion in the image.

Thresholds = [0,5,10,15,20,25,30]

As the threshold value increases, the motion of the objects and the object itself gets insignificant. Even low threshold values do not show significant motion. Threshold value of 25 seems optimal.

Gammas = [0, 0.03, 0.06, 0.09, 0.12, 1]

As the value of gamma decreases, the history of the image is more clearly seen. Noticeably, gamma value of 0.09 is ideal. As the value of gamma increases, the fading effect is insignificant with the patches of previous frame opaque.

In [3]:
# Persistent Backgrounding
def persistentFD(inputfile, threshold, gamma):
    cap = video.create_capture(inputfile)
    ret, first = cap.read()

    # For background subtraction, Save the first image as reference
    first_gray = cv2.cvtColor(first, cv2.COLOR_BGR2GRAY)
    first_gray = cv2.GaussianBlur(first_gray, (21, 21), 0)
    
    backgroundFrame = first_gray
    
    h, w = first_gray.shape[:2]
    start = time.time()
    mh = np.zeros((h, w),np.float32)
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
    
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (21, 21), 0)
        currentFrame = gray
        
        foreground = cv2.absdiff(backgroundFrame, currentFrame)

        foreground = cv2.threshold(foreground, threshold, 255, cv2.THRESH_BINARY)[1]
        foreground = cv2.dilate(foreground, None, iterations = 5)
        
        mh = mh - gamma
        mh[mh < 0] = 0
        tmp = mh
        
        foreground = 255 * foreground
        for i in range(h):
            for j in range(w):
                
                if foreground[i, j] > tmp[i, j]:
                    mh[i,j] = foreground[i, j]
                else:
                    mh[i,j] = tmp[i, j] 
        
        cv2.imshow("Persistent frame differencing on {0} with threshold = {1} gamma = {2}".format(inputfile, threshold, gamma) ,mh)
        backgroundFrame = currentFrame
        
        key = cv2.waitKey(1) & 0xFF 
    
        # if the `q` key is pressed, break from the loop
        if key == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()
    print('Time taken for threshold {0}: '.format(threshold) + str(time.time() - start))

In [7]:
# Ideal case
persistentFD('beach.avi', 10, 0.09)

Time taken for threshold 10: 7.293838262557983


In [42]:
gammas = [0, 0.03, 0.06, 0.09, 0.12, 1]
thresholds = [0, 5,10,15,20,25,30]
sequences = ['beach.avi', 'trees.avi', 'lights.avi', 'jug.avi', 'railway.avi', 'rock.avi']
for sequence in sequences:
    for threshold in thresholds:    
        for gamma in gammas:
            persistentFD(sequence,threshold, gamma)

Time taken for threshold 0: 80.92021751403809
Time taken for threshold 0: 86.73180937767029


KeyboardInterrupt: 